In [1]:
import numpy as np 
import matplotlib.pylab as plt 
import scipy.io
import pandas as pd

def to_df(mat_db):
    """Returns one pd.DataFrame per cycle type"""

    # Features common for every cycle
    cycles_cols = ['type', 'ambient_temperature', 'time']

    # Features monitored during the cycle
    features_cols = {
        'charge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                   'Current_charge', 'Voltage_charge', 'Time'],
        'discharge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                      'Current_charge', 'Voltage_charge', 'Time', 'Capacity'],
        'impedance': ['Sense_current', 'Battery_current', 'Current_ratio',
                      'Battery_impedance', 'Rectified_impedance', 'Re', 'Rct']
    }

    # Define one pd.DataFrame per cycle type
    df = {key: pd.DataFrame() for key in features_cols.keys()}

    # Get every cycle
    cycles = [[row.flat[0] for row in line] for line in mat_db[0][0][0][0]]

    # Get measures for every cycle
    for cycle_id, cycle_data in enumerate(cycles):
        tmp = pd.DataFrame()

        # Data series for every cycle
        features_x_cycle = cycle_data[-1]

        # Get features for the specific cycle type
        features = features_cols[cycle_data[0]]
        
        for feature, data in zip(features, features_x_cycle):
            if len(data[0]) > 1:
                # Correct number of records
                tmp[feature] = data[0]
            else:
                # Single value, so assign it to all rows
                tmp[feature] = data[0][0]
        
        # Add columns common to the cycle measurements
        tmp['id_cycle'] = cycle_id
        for k, col in enumerate(cycles_cols):
            tmp[col] = cycle_data[k]
        
        # Append cycle data to the right pd.DataFrame using pd.concat()
        cycle_type = cycle_data[0]
        df[cycle_type] = pd.concat([df[cycle_type], tmp], ignore_index=True)
    
    return df


def Mat2List(dfs_mat):
    # Example usage
    dfs_B0005 = to_df(dfs_mat)

    df_cycle_charge = dfs_B0005['charge'] #['id_cycle']
    df_cycle_dicharge = dfs_B0005['discharge'] #['id_cycle']
    
    total_result = []

    for i in df_cycle_charge['id_cycle'].unique():
        # Filter charge data for the current cycle
        df = df_cycle_charge[df_cycle_charge['id_cycle'] == i]

        # Extract the required columns
        temperature = df['Temperature_measured'].tolist() # 이 리스트의 MinMaxScaler를 통해 정규화
        current = df['Current_measured'].tolist()
        voltage = df['Voltage_measured'].tolist()

        # Find corresponding discharge data
        dis = df_cycle_dicharge[df_cycle_dicharge['id_cycle'] == i + 1]
        
        # Fallback to next cycle if discharge data is empty
        if dis.empty:
            dis = df_cycle_dicharge[df_cycle_dicharge['id_cycle'] == i + 2]

        # Calculate the label (mean capacity), handle if still empty
        label = dis['Capacity'].mean() if not dis.empty else None
        

        # Skip if label is None
        if label is None:
            continue

        else:
        
            #result = [[np.array(temperature), np.array(current), np.array(voltage)], label]
            result = [np.array(temperature), np.array(current), np.array(voltage)], label
            # result = np.array(np.array(zip(temperature, current, voltage)), label)
            total_result.append(result)
        del result
    # Check the resulting dataset
    print(f"Total results: {len(total_result)}")

    return total_result

B0005 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0005.mat')
B0006 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0006.mat')
B0007 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0007.mat')
B0018 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0018.mat')

B0005 = B0005['B0005']
B0006 = B0006['B0006']
B0007 = B0007['B0007']
B0018 = B0018['B0018']
# Example usage
dfs_B0005 = to_df(B0005)
dfs_B0006 = to_df(B0006)
dfs_B0007 = to_df(B0007)
dfs_B0018 = to_df(B0018)

batt_list = [B0005,B0006,B0007]

df_train = []
for i in batt_list:
    df_train+=Mat2List(i)

df_test = Mat2List(B0018)

Total results: 169
Total results: 169
Total results: 169
Total results: 133


In [2]:
import numpy as np
import pandas as pd
import scipy.io
from sklearn.preprocessing import MinMaxScaler


def to_df(mat_db):
    """Returns one pd.DataFrame per cycle type."""
    cycles_cols = ['type', 'ambient_temperature', 'time']
    features_cols = {
        'charge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                   'Current_charge', 'Voltage_charge', 'Time'],
        'discharge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                      'Current_charge', 'Voltage_charge', 'Time', 'Capacity'],
        'impedance': ['Sense_current', 'Battery_current', 'Current_ratio',
                      'Battery_impedance', 'Rectified_impedance', 'Re', 'Rct']
    }

    df = {key: pd.DataFrame() for key in features_cols.keys()}
    cycles = [[row.flat[0] for row in line] for line in mat_db[0][0][0][0]]

    for cycle_id, cycle_data in enumerate(cycles):
        tmp = pd.DataFrame()
        features_x_cycle = cycle_data[-1]
        cycle_type = cycle_data[0]

        if cycle_type not in features_cols:
            continue

        features = features_cols[cycle_type]
        for feature, data in zip(features, features_x_cycle):
            if len(data[0]) > 1:
                tmp[feature] = data[0]
            else:
                tmp[feature] = [data[0][0]] * len(tmp)

        tmp['id_cycle'] = cycle_id
        for k, col in enumerate(cycles_cols):
            tmp[col] = cycle_data[k]

        df[cycle_type] = pd.concat([df[cycle_type], tmp], ignore_index=True)

    return df


def Mat2List(dfs_mat):
    dfs = to_df(dfs_mat)

    df_cycle_charge = dfs['charge']
    df_cycle_discharge = dfs['discharge']

    total_result = []
    scaler = MinMaxScaler(feature_range=(0, 1))  # Initialize MinMaxScaler

    for i in df_cycle_charge['id_cycle'].unique():
        df = df_cycle_charge[df_cycle_charge['id_cycle'] == i]

        # Extract required columns and apply MinMax scaling
        temperature = np.array(df['Temperature_measured'].tolist()).reshape(-1, 1)
        current = np.array(df['Current_measured'].tolist()).reshape(-1, 1)
        voltage = np.array(df['Voltage_measured'].tolist()).reshape(-1, 1)

        normalized_temperature = scaler.fit_transform(temperature).flatten()
        normalized_current = scaler.fit_transform(current).flatten()
        normalized_voltage = scaler.fit_transform(voltage).flatten()

        # Find corresponding discharge data
        dis = df_cycle_discharge[df_cycle_discharge['id_cycle'] == i + 1]
        if dis.empty:
            dis = df_cycle_discharge[df_cycle_discharge['id_cycle'] == i + 2]

        # Calculate label
        label = dis['Capacity'].mean() if not dis.empty else None

        if label is None:
            continue

        # result = [[normalized_temperature, normalized_current, normalized_voltage], label]
        result = [normalized_temperature, normalized_current, normalized_voltage], label
        total_result.append(result)

    print(f"Total results: {len(total_result)}")
    return total_result


# Load MATLAB files
B0005 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0005.mat')['B0005']
B0006 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0006.mat')['B0006']
B0007 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0007.mat')['B0007']
B0018 = scipy.io.loadmat('./DATA/1. BatteryAgingARC-FY08Q4/B0018.mat')['B0018']

# Process data
batt_list = [B0005, B0006, B0007]
df_train = []
for batt in batt_list:
    df_train += Mat2List(batt)

df_test = Mat2List(B0018)

Total results: 169
Total results: 169
Total results: 169
Total results: 133


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# Dataset 정의
class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence, label = self.data[idx]
        sequence = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)  # Channel dimension 추가
        label = torch.tensor(label, dtype=torch.float32)
        return sequence, label
    
def collate_fn(batch):
    sequences, labels = zip(*batch)
    # 가장 긴 길이 계산 (마지막 차원 기준)
    max_length = max(seq.shape[-1] for seq in sequences)
    # 각 텐서에 대해 패딩 적용
    padded_sequences = [
        torch.nn.functional.pad(seq, (0, max_length - seq.shape[-1]))  # 마지막 차원을 기준으로 패딩
        for seq in sequences
    ]
    # 배치를 쌓기
    padded_sequences = torch.stack(padded_sequences)
    # 레이블 배치 생성
    labels = torch.tensor(labels, dtype=torch.float32)
    return padded_sequences, labels

# 데이터 클리닝 함수
def clean_data(dataset):
    cleaned_data = []
    for sequence, label in dataset:
        # NaN이 없는 데이터만 추가
        if not (torch.isnan(sequence).any() or torch.isnan(label).any()):
            cleaned_data.append((sequence.numpy(), label.item()))  # Python 기본 타입으로 변환
    return cleaned_data

# 샘플 데이터 (train과 val 데이터셋 정의)

# Dataset 객체 생성
train_dataset_with_nan = SequenceDataset(df_train)
val_dataset_with_nan = SequenceDataset(df_test)

# NaN 데이터 제거
cleaned_train_data = clean_data(train_dataset_with_nan)
cleaned_val_data = clean_data(val_dataset_with_nan)

# NaN 제거된 데이터셋 래핑
cleaned_train_dataset = SequenceDataset(cleaned_train_data)
cleaned_val_dataset = SequenceDataset(cleaned_val_data)

# DataLoader 생성
train_loader = DataLoader(cleaned_train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(cleaned_val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

C:\Users\ime203\AppData\Local\Temp\ipykernel_7380\1006805831.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  sequence = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)  # Channel dimension 추가


In [3]:
train_dataset_with_nan[0]

(tensor([[[ 2.4655e+01,  2.4666e+01,  2.4675e+01,  ...,  2.4519e+01,
            2.4514e+01,  2.4507e+01],
          [-1.2007e-03, -4.0303e+00,  1.5127e+00,  ..., -3.5059e-04,
           -1.8559e-03, -2.8924e-03],
          [ 3.8730e+00,  3.4794e+00,  4.0006e+00,  ...,  4.1914e+00,
            4.1915e+00,  4.1911e+00]]]),
 tensor(1.8565))

In [105]:
import torch.nn as nn

class RegressionRNN(nn.Module):
    def __init__(self,input_shape):
        super(RegressionRNN, self).__init__()
        self.rnn = nn.LSTM(input_size=input_shape[0], hidden_size=16, batch_first=True)
        self.fc = nn.Linear(16, 1)
    
    def forward(self, x):
        _, (hidden, _) = self.rnn(x)  # hidden -> (1, batch_size, 16)
        x = hidden.squeeze(0)  # (batch_size, 16)
        x = self.fc(x)  # (batch_size, 1)
        return x
    

import torch.optim as optim
input_shape = (1, 3, 3828)  # Example input shape from train_loader


# 모델, 손실 함수 및 옵티마이저 정의
model = RegressionRNN(input_shape[1:])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
for epoch in range(50):  # 에포크 수
    for features, labels in train_loader:
        # Forward
        outputs = model(features)  # (batch_size, 1)
        loss = criterion(outputs.squeeze(), labels)  # (batch_size,)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


ValueError: LSTM: Expected input to be 2D or 3D, got 5D instead

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=input_shape[0], out_channels=30, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))

        self.conv2 = nn.Conv2d(in_channels=30, out_channels=15, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu2 = nn.LeakyReLU(0.1)

        # Dynamically adjust the pooling size for valid output dimensions
        self.pool2 = nn.AdaptiveMaxPool2d((1, 1))

        # Calculate the flatten size dynamically
        self.flatten_size = self._get_flatten_size(input_shape)

        self.fc = nn.Linear(self.flatten_size, 1)

    def _get_flatten_size(self, input_shape):
        x = torch.zeros(1, *input_shape)  # Simulate a batch with size 1
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)
        return x.numel()
    def forward(self, x):
        # Remove unnecessary dimensions (squeeze depth dimension if it's 1)
        x = x.squeeze(2)  # Assuming input is [batch_size, channels, depth=1, height, width]
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x


# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example usage
input_shape = (1, 3, 3828)  # Example input shape from train_loader
model = CNNModel(input_shape[1:]).to(device)

# Print model summary
print(model)

# Training parameters
epochs = 200
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for features, labels in train_loader:
        # 데이터 이동
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels.unsqueeze(1))  # Labels 차원 맞추기

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training loss
    train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_features, val_labels in val_loader:
            val_features, val_labels = val_features.to(device), val_labels.to(device)
            val_outputs = model(val_features)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    # Print losses for this epoch
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Testing loop
test_loss = 0.0
with torch.no_grad():
    for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()

test_loss /= len(val_loader)
print(f"Test Loss: {test_loss:.4f}")


Using device: cuda


RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [1, 3, 3828]

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=30, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.dropout1 = nn.Dropout(0.1)  # Dropout layer with 0.1 rate

        self.conv2 = nn.Conv2d(in_channels=30, out_channels=15, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu2 = nn.LeakyReLU(0.1)
        self.dropout2 = nn.Dropout(0.1)  # Dropout layer with 0.1 rate

        # Dynamically adjust the pooling size for valid output dimensions
        self.pool2 = nn.AdaptiveMaxPool2d((1, 1))

        # Calculate the flatten size dynamically
        self.flatten_size = self._get_flatten_size(input_shape)

        self.fc = nn.Linear(self.flatten_size, 1)
        self.dropout_fc = nn.Dropout(0.1)  # Dropout before the fully connected layer

    def _get_flatten_size(self, input_shape):
        x = torch.zeros(1, *input_shape)  # Simulate a batch with size 1
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)
        return x.numel()

    def forward(self, x):
        # Remove unnecessary dimensions (squeeze depth dimension if it's 1)
        x = x.squeeze(2)  # Assuming input is [batch_size, channels, depth=1, height, width]
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.dropout1(x)  # Apply dropout after first pooling

        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.dropout2(x)  # Apply dropout after second activation

        x = self.pool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.dropout_fc(x)  # Apply dropout before the fully connected layer
        x = self.fc(x)
        return x

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example usage
input_shape = (1, 1, 3, 3828)  # Example input shape from train_loader
model = CNNModel(input_shape[1:]).to(device)

# Print model summary
print(model)

# Training parameters
epochs = 200
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for features, labels in train_loader:
        # 데이터 이동
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels.unsqueeze(1))  # Labels 차원 맞추기

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training loss
    train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_features, val_labels in val_loader:
            val_features, val_labels = val_features.to(device), val_labels.to(device)
            val_outputs = model(val_features)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    # Print losses for this epoch
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Testing loop
test_loss = 0.0
with torch.no_grad():
    for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()

test_loss /= len(val_loader)
print(f"Test Loss: {test_loss:.4f}")

Using device: cuda
CNNModel(
  (conv1): Conv2d(1, 30, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu1): LeakyReLU(negative_slope=0.1)
  (pool1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.1, inplace=False)
  (conv2): Conv2d(30, 15, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu2): LeakyReLU(negative_slope=0.1)
  (dropout2): Dropout(p=0.1, inplace=False)
  (pool2): AdaptiveMaxPool2d(output_size=(1, 1))
  (fc): Linear(in_features=15, out_features=1, bias=True)
  (dropout_fc): Dropout(p=0.1, inplace=False)
)
Epoch 1/100, Train Loss: 0.1171, Val Loss: 0.3602
Epoch 2/100, Train Loss: 0.0718, Val Loss: 0.3226
Epoch 3/100, Train Loss: 0.0620, Val Loss: 0.3239
Epoch 4/100, Train Loss: 0.0666, Val Loss: 0.3722
Epoch 5/100, Train Loss: 0.0619, Val Loss: 0.2773
Epoch 6/100, Train Loss: 0.0597, Val Loss: 0.3121
Epoch 7/100, Train Loss: 0.0492, Val Loss: 0.2112
Epoch 8/100, Train Loss: 0.0496, Val Loss: 0.1973
Epoch 9/100, Trai

In [128]:
import torch
import torch.nn as nn

# CNN + LSTM 모델 정의
class CNNLSTMModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNLSTMModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=16, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu2 = nn.LeakyReLU(0.1)
        self.pool2 = nn.AdaptiveMaxPool2d((1, None))  # 유지된 sequence dimension

        self.lstm = nn.LSTM(input_size=16, hidden_size=16, num_layers=1, batch_first=True)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        # 5D 텐서 입력을 4D로 변환
        x = x.squeeze(2)  # Remove depth dimension -> (batch_size, channels, height, width)

        # CNN 통과
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)  # Output shape: (batch_size, channels=15, height=1, sequence_length)

        # 차원 변환: (batch_size, sequence_length, features)
        x = x.squeeze(2).permute(0, 2, 1)  # Remove height dimension and permute

        # LSTM 통과
        lstm_out, _ = self.lstm(x)  # -> (batch_size, sequence_length, hidden_size)
        x = lstm_out[:, -1, :]  # 마지막 time step의 hidden state 사용

        # FC 계층
        x = self.fc(x)  # -> (batch_size, 1)
        return x



# Example usage
input_shape = (1, 1, 3, 3828)  # Example input shape from train_loader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNLSTMModel(input_shape[1:]).to(device)

# Print model summary
print(model)

# Training parameters
epochs = 100
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels.unsqueeze(1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training loss
    train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_features, val_labels in val_loader:
            val_features, val_labels = val_features.to(device), val_labels.to(device)
            val_outputs = model(val_features)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    # Print losses for this epoch
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Testing loop
test_loss = 0.0
with torch.no_grad():
    for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()

test_loss /= len(val_loader)
print(f"Test Loss: {test_loss:.4f}")


CNNLSTMModel(
  (conv1): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu1): LeakyReLU(negative_slope=0.1)
  (pool1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu2): LeakyReLU(negative_slope=0.1)
  (pool2): AdaptiveMaxPool2d(output_size=(1, None))
  (lstm): LSTM(16, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)
Epoch 1/100, Train Loss: 0.4631, Val Loss: 0.0245
Epoch 2/100, Train Loss: 0.0457, Val Loss: 0.0270
Epoch 3/100, Train Loss: 0.0427, Val Loss: 0.0281
Epoch 4/100, Train Loss: 0.0418, Val Loss: 0.0274
Epoch 5/100, Train Loss: 0.0439, Val Loss: 0.0277
Epoch 6/100, Train Loss: 0.0425, Val Loss: 0.0321
Epoch 7/100, Train Loss: 0.0427, Val Loss: 0.0271
Epoch 8/100, Train Loss: 0.0428, Val Loss: 0.0275
Epoch 9/100, Train Loss: 0.0432, Val Loss: 0.0276
Epoch 10/100, Train Loss: 0.0421, Val Loss: 0.0303
Epoch 11/100, Train L

In [118]:
import torch
import math

# Testing loop with RMSE and MAPE calculation
model.eval()
test_loss = 0.0
predictions = []
actuals = []

with torch.no_grad():
    for features, labels in val_loader:
        # Move data to the same device as the model
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)

        # Collect predictions and actual values
        predictions.extend(outputs.view(-1).tolist())
        actuals.extend(labels.view(-1).tolist())

        # Compute batch loss
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()

# Compute final RMSE
test_loss /= len(val_loader)
rmse = math.sqrt(sum((p - a) ** 2 for p, a in zip(predictions, actuals)) / len(actuals))

# Compute MAPE
mape = sum(abs((p - a) / a) for p, a in zip(predictions, actuals) if a != 0) / len(actuals) * 100

# Print results
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAPE: {mape:.2f}%")

Test Loss (MSE): 0.0257
Test RMSE: 0.1604
Test MAPE: 8.43%


In [ ]:
Test Loss (MSE): 0.2022
Test RMSE: 0.4496
Test MAPE: 26.73%

In [ ]:
import torch
import math

# Testing loop with RMSE and MAPE calculation
model.eval()
test_loss = 0.0
predictions = []
actuals = []

with torch.no_grad():
    for features, labels in val_loader:
        # Move data to the same device as the model
        features, labels = features.to(device), labels.to(device)
        features = features.squeeze(2)

        # Forward pass
        outputs = model(features)

        # Collect predictions and actual values
        predictions.extend(outputs.view(-1).tolist())
        actuals.extend(labels.view(-1).tolist())

        # Compute batch loss
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()

# Compute final RMSE
test_loss /= len(val_loader)
rmse = math.sqrt(sum((p - a) ** 2 for p, a in zip(predictions, actuals)) / len(actuals))

# Compute MAPE
mape = sum(abs((p - a) / a) for p, a in zip(predictions, actuals) if a != 0) / len(actuals) * 100

# Print results
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAPE: {mape:.2f}%")